# PI-II: Demo QRF

A simple example of fitting a quantile regression forest (QRF) using the [quantile-forest](https://zillow.github.io/quantile-forest/) package to estimate prediction intervals.

Starter script for the [Prediction interval competition II: House price](https://www.kaggle.com/competitions/prediction-interval-competition-ii-house-price/overview) competition.

In [1]:
!pip install -q quantile-forest 2>/dev/null  # package for quantile regression forests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 77.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from quantile_forest import RandomForestQuantileRegressor
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from typing import Tuple

random_state = 0
np.random.seed(random_state)

# Competition variables.
base_path = "/kaggle/input/prediction-interval-competition-ii-house-price/"
alpha = 0.1  # the specified competition alpha (i.e., 90% coverage)

In [3]:
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    """Compute the Winkler Interval Score for prediction intervals.

    Args:
        y_true (array-like): True observed values.
        lower (array-like): Lower bounds of prediction intervals.
        upper (array-like): Upper bounds of prediction intervals.
        alpha (float): Significance level (e.g., 0.1 for 90% intervals).
        return_coverage (bool): If True, also return empirical coverage.

    Returns:
        score (float): Mean Winkler Score.
        coverage (float, optional): Proportion of true values within intervals.
    """
    y_true = np.asarray(y_true)
    lower = np.asarray(lower)
    upper = np.asarray(upper)

    width = upper - lower
    penalty_lower = 2 / alpha * (lower - y_true)
    penalty_upper = 2 / alpha * (y_true - upper)

    score = width.copy()
    score += np.where(y_true < lower, penalty_lower, 0)
    score += np.where(y_true > upper, penalty_upper, 0)

    if return_coverage:
        inside = (y_true >= lower) & (y_true <= upper)
        coverage = np.mean(inside)
        return np.mean(score), coverage

    return np.mean(score)

In [4]:
df = pd.read_csv(base_path + "dataset.csv", index_col="id", parse_dates=["sale_date"])
df_test = pd.read_csv(base_path + "test.csv", index_col="id", parse_dates=["sale_date"])

## Data Preparation

Prepare the data, including separating features and target, encoding categoricals, imputation, and simple feature engineering.

In [5]:
# Split features and target.
X = df.drop("sale_price", axis=1)
y = df["sale_price"]

# Split train/val and test.
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.50, random_state=random_state
)
X_test = df_test.copy()

In [6]:
# Ordinal encoding.
cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X_train[cat_cols] = encoder.fit_transform(X_train[cat_cols])
X_val[cat_cols] = encoder.transform(X_val[cat_cols])
X_test[cat_cols] = encoder.transform(X_test[cat_cols])

In [7]:
# Imputation.
num_cols = X_train.select_dtypes(include="number").columns.tolist()

num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")


def impute(df, cols, imputer, fit=False):
    """Helper function for imputation."""
    if fit:
        return pd.DataFrame(imputer.fit_transform(df[cols]), columns=cols, index=df.index)
    else:
        return pd.DataFrame(imputer.transform(df[cols]), columns=cols, index=df.index)


X_train[num_cols] = impute(X_train, num_cols, num_imputer, fit=True)
X_val[num_cols] = impute(X_val, num_cols, num_imputer)
X_test[num_cols] = impute(X_test, num_cols, num_imputer)

X_train[cat_cols] = impute(X_train, cat_cols, cat_imputer, fit=True)
X_val[cat_cols] = impute(X_val, cat_cols, cat_imputer)
X_test[cat_cols] = impute(X_test, cat_cols, cat_imputer)

In [8]:
class SaleDateEncoder(BaseEstimator, TransformerMixin):
    """Encode sale date as a week of the year feature."""

    def __init__(self, date_column="sale_date"):
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.assign(
            **{
                "sale_week": lambda x: x["sale_date"].dt.isocalendar().week,
            }
        ).drop(columns=["sale_date"])
        return X

    def fit_transform(self, X):
        return self.fit(self, X).transform(X)


# Sale date encoding.
saledate_encoder = SaleDateEncoder(date_column="sale_date")
X_train = saledate_encoder.fit_transform(X_train)
X_val = saledate_encoder.transform(X_val)
X_test = saledate_encoder.transform(X_test)

## Model Fitting

Fit a QRF model and use it to estimate a nominal marginal coverage of 90% (quantiles 0.05 and 0.95).

In [9]:
class GBDTIntervalRegressor(BaseEstimator, RegressorMixin):
    """
    基于 scikit-learn GBDT / LightGBM 的分位数回归器，
    支持同时训练多个分位数模型并输出区间。
    """

    def __init__(
        self,
        backend: str = "sklearn",     # 或 "lightgbm"
        n_estimators: int = 100,
        learning_rate: float = 0.1,
        max_depth: int = 3,
        quantiles: Tuple[float, float, float] = (0.05, 0.5, 0.95),
        **kwargs
    ):
        self.backend = backend
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.quantiles = quantiles
        self.kwargs = kwargs
        self.models_ = {}  # 存放 q 下界、中位数、上界三棵树

    def fit(self, X, y):
        """
        训练三个子模型：下分位数、中位数、上分位数
        """
        for q in self.quantiles:
            if self.backend == "sklearn":
                model = GradientBoostingRegressor(
                    loss="quantile", alpha=q,
                    n_estimators=self.n_estimators,
                    learning_rate=self.learning_rate,
                    max_depth=self.max_depth,
                    **self.kwargs
                )
            else:  # lightgbm
                params = dict(
                    objective="quantile",
                    alpha=q,
                    n_estimators=self.n_estimators,
                    learning_rate=self.learning_rate,
                    max_depth=self.max_depth,
                    **self.kwargs
                )
                model = lgb.LGBMRegressor(**params)
            model.fit(X, y)
            self.models_[q] = model
        return self

    def predict(self, X) -> np.ndarray:
        lower = self.models_[self.quantiles[0]].predict(X)
        upper = self.models_[self.quantiles[2]].predict(X)
        # 返回一个 (n_samples, 2) 的矩阵
        return np.stack((lower, upper), axis=1)


    @property
    def feature_importances_(self):
        """返回各个模型的重要性（可以取平均或单独分析）。"""
        imps = {q: m.feature_importances_ for q, m in self.models_.items()}
        return imps

    def get_params(self, deep=True):
        """支持 sklearn 参数搜索"""
        return {
            "backend": self.backend,
            "n_estimators": self.n_estimators,
            "learning_rate": self.learning_rate,
            "max_depth": self.max_depth,
            "quantiles": self.quantiles,
            **self.kwargs,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self

In [10]:
model = GBDTIntervalRegressor().fit(X_train, y_train)

In [11]:
y_val_pred = model.predict(X_val)
y_val_pred = pd.DataFrame(y_val_pred, columns=["pi_lower", "pi_upper"])
#产生每个样本的预测区间并保存

## Evaluation

Evaluate the QRF predictions on the validation data.

In [12]:
mws, coverage = winkler_score(
    y_val,
    y_val_pred["pi_lower"],
    y_val_pred["pi_upper"],
    alpha=alpha,
    return_coverage=True,
)

print("Mean Winkler Score:", round(mws, 2))
print("Coverage:", round(coverage * 100, 1), "%")

Mean Winkler Score: 901740.02
Coverage: 89.8 %


## Submission

In [13]:
# Predict intervals on test set.
test_preds = model.predict(X_test)

sample_submission = pd.read_csv(base_path + "sample_submission.csv")
sample_submission["pi_lower"] = test_preds[:, 0]
sample_submission["pi_upper"] = test_preds[:, 1]
sample_submission.to_csv("submission.csv",
                         index=False,
                         float_format="%.6f")

sample_submission

,id,pi_lower,pi_upper
0,200000,553799.744720,1.056133e+06
1,200001,628734.264791,1.844173e+06
2,200002,331887.387839,1.299444e+06
3,200003,216573.992742,4.376405e+05
4,200004,271605.456153,1.309509e+06
...,...,...,...
199995,399995,219653.639384,8.934583e+05
199996,399996,239270.515570,8.018955e+05
199997,399997,194613.374663,5.396006e+05
199998,399998,390722.163434,1.424492e+06
